In [41]:
from pathlib import Path
import pandas as pd
IN_PATH = Path('../../data/raw_data/ACSPUMS_RAW.csv')
OUT_PATH = Path('../../data/processed_data/ACSPUMS_CLEANED.csv')

# Load Data

In [42]:
df = pd.read_csv(IN_PATH)
print(df.shape)
df.head(5)

(216151, 15)


,PWGTP,WGTP,NATIVITY,NOP,CIT,LANX,SCHL,LNGI,AGEP,POVPIP,PWGTP.1,NRC,GRPIP,STATE,Unnamed: 14
0,24,0,1,0,1,1,18,0,44,-1,24,-1,0,12,NaN
1,93,0,1,0,1,2,12,0,15,-1,93,-1,0,12,NaN
2,19,0,1,0,1,2,16,0,69,0,19,-1,0,12,NaN
3,66,0,1,0,1,2,19,0,20,-1,66,-1,0,12,NaN
4,84,0,1,0,1,2,11,0,14,-1,84,-1,0,12,NaN


# Dropping columns

Dropping duplicated and junk columns

In [43]:
columns_to_drop = ['PWGTP.1', 'Unnamed: 14']
df_working = df.drop(columns=columns_to_drop, errors = 'ignore').copy()
print(df_working.shape)
df_working.head(10)

(216151, 13)


,PWGTP,WGTP,NATIVITY,NOP,CIT,LANX,SCHL,LNGI,AGEP,POVPIP,NRC,GRPIP,STATE
0,24,0,1,0,1,1,18,0,44,-1,-1,0,12
1,93,0,1,0,1,2,12,0,15,-1,-1,0,12
2,19,0,1,0,1,2,16,0,69,0,-1,0,12
3,66,0,1,0,1,2,19,0,20,-1,-1,0,12
4,84,0,1,0,1,2,11,0,14,-1,-1,0,12
5,81,0,1,0,1,2,18,0,20,-1,-1,0,12
6,21,0,1,0,1,2,16,0,56,39,-1,0,12
7,43,0,1,0,1,2,21,0,34,-1,-1,0,12
8,48,0,1,0,1,2,19,0,88,-1,-1,0,12
9,58,0,1,0,1,2,16,0,76,-1,-1,0,12


## Filter by Age

Filter the population by age

In [44]:
if 'df_working' in locals():
    AGE_CUTOFF = 25
    df_working = df_working[df_working['AGEP'] >= AGE_CUTOFF].copy()

# Haddling Missing Or Not Applicable Codes

We filter out rows where (NATIVITY, SCHL, POVPIP) the key variables contain the missing code, which is -1 and 'Not in Universe' coded as 0

In [45]:
CORE_COLS = ['NATIVITY', 'SCHL', 'POVPIP']
MISSING_CODE = -1
NOT_IN_UNIVERSE_CODE = 0

for col in CORE_COLS:
    df_working = df_working[df_working[col] != MISSING_CODE]

In [46]:
df_working = df_working[df_working['SCHL'] != NOT_IN_UNIVERSE_CODE]
print(df_working.shape)

(159869, 13)


# Feature Engineering

Creating the dummy variable for immigrant generation

In [47]:
FOREIGN_BORN = 2

df_working['Immigrant_Generation'] = df_working['NATIVITY'].apply(
    lambda x: 'Immigrant (Foreign-Born)' if x == FOREIGN_BORN else 'U.S. Born'
)

print("\n--- Corrected Immigrant Generation Counts (Binary) ---")
print(df_working['Immigrant_Generation'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%')

df_working['Is_Immigrant'] = df_working['Immigrant_Generation'].apply(
    lambda x: 1 if x == 'Immigrant (Foreign-Born)' else 0
)


--- Corrected Immigrant Generation Counts (Binary) ---
Immigrant_Generation
U.S. Born                   75.69%
Immigrant (Foreign-Born)    24.31%
Name: proportion, dtype: object


In [53]:
import numpy as np

conditions = [
    
    (df_working['SCHL'] >= 1) & (df_working['SCHL'] <= 11), 
    
    (df_working['SCHL'] >= 12) & (df_working['SCHL'] <= 16),
    
    (df_working['SCHL'] == 17) | (df_working['SCHL'] == 18),
    
    (df_working['SCHL'] == 19),
    
    (df_working['SCHL'] == 20),
    
    (df_working['SCHL'] == 21),
    
    (df_working['SCHL'] == 22),
    
    (df_working['SCHL'] >= 23) & (df_working['SCHL'] <= 24)
]

choices = [
    df_working['SCHL'],
    12,
    13,
    14,
    16,
    18,
    19, 
    20 
]

df_working['Years_of_Schooling'] = np.select(conditions, choices, default=np.nan)

print("\n--- Verification of Outcome Variable (Y) ---")

mean_years = df_working.groupby('Immigrant_Generation')['Years_of_Schooling'].mean().round(2)
print("Mean Years of Schooling by Generation:")
print(mean_years)
print(f"Total records analyzed (must not have NaN): {df_working['Years_of_Schooling'].count()}")


--- Verification of Outcome Variable (Y) ---
Mean Years of Schooling by Generation:
Immigrant_Generation
Immigrant (Foreign-Born)    14.35
U.S. Born                   15.13
Name: Years_of_Schooling, dtype: float64
Total records analyzed (must not have NaN): 159869


In [ ]:
conditions = [
    
    (df_working['LANX'] == 2) 
]
choices = [1]
default_value = 0 

df_working['Speaks_English_Only'] = np.select(conditions, choices, default=default_value)

print("Speaks_English_Only variable created.")

Speaks_English_Only variable created.


In [51]:
df_working.to_csv(OUT_PATH, index=False, encoding='utf-8')

print(f"✅ Clean dataset successfully saved to: {OUT_PATH}")

✅ Clean dataset successfully saved to: ..\..\data\processed_data\ACSPUMS_CLEANED.csv
